In [54]:
import pandas as pd
import numpy as np
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
#import geopandas as gpd
import geopy as gp
from geopy.geocoders import GoogleV3
from tqdm import tqdm
import datapane as dp
import plotly.graph_objects as go

In [55]:
df = pd.read_csv('japan_df_province')

In [56]:
#df['Date'] = df['Captured Time'].apply(lambda x: x.year)
df['Date'] = pd.to_datetime(df['Captured Time'], format='%Y-%m-%d %H:%M:%S')
df.drop('Captured Time', axis=1, inplace=True)

In [57]:
df = df[df['Date'].notnull()]
df = df.sort_values('Date')

In [58]:
df.dtypes


Unnamed: 0               int64
Unnamed: 0.1             int64
Latitude               float64
Longitude              float64
Value                  float64
Height                 float64
Country                 object
Province                object
Date            datetime64[ns]
dtype: object

In [59]:
df

,Unnamed: 0,Unnamed: 0.1,Latitude,Longitude,Value,Height,Country,Province,Date
14671489,18590243,35803309,37.508102,139.932585,33.0,1.0,Japan,Fukushima Prefecture,2012-12-02 05:59:30
21294384,25351193,35731757,37.772805,140.482043,239.0,0.0,Japan,Fukushima Prefecture,2014-10-20 12:49:58
21294383,25351192,35731753,37.771039,140.474200,249.0,0.0,Japan,Fukushima Prefecture,2014-10-20 12:59:36
21294382,25351191,35731750,37.760245,140.470765,452.0,0.0,Japan,Fukushima Prefecture,2014-10-20 13:20:44
22771921,26828744,35731747,37.930088,140.762162,350.0,0.0,Japan,Miyagi Prefecture,2014-10-20 15:33:58
...,...,...,...,...,...,...,...,...,...
0,295606,12,31.833193,130.301922,15.0,13.0,Japan,Kagoshima Prefecture,2022-03-19 01:58:23
13749458,17667570,7,37.496139,140.986419,123.0,10.0,Japan,Fukushima Prefecture,2022-03-19 01:59:09
17109706,21053975,6,37.563500,140.991700,40.0,5.0,Japan,Fukushima Prefecture,2022-03-19 01:59:09
13749457,17667569,5,37.496139,140.986419,43.0,10.0,Japan,Fukushima Prefecture,2022-03-19 01:59:15


In [60]:
df = df.groupby([pd.Grouper(key='Date', freq='D'), 'Province']).mean().reset_index()


In [61]:
df

,Date,Province,Unnamed: 0,Unnamed: 0.1,Latitude,Longitude,Value,Height
0,2012-12-02,Fukushima Prefecture,1.859024e+07,3.580331e+07,37.508102,139.932585,33.000000,1.000000
1,2014-10-20,Fukushima Prefecture,2.535119e+07,3.573175e+07,37.768030,140.475669,313.333333,0.000000
2,2014-10-20,Miyagi Prefecture,2.682874e+07,3.573175e+07,37.930088,140.762162,350.000000,0.000000
3,2015-07-09,no data,7.848781e+06,3.558452e+07,35.659491,139.727850,19.767742,92.000000
4,2015-07-10,no data,7.848267e+06,3.558401e+07,35.659491,139.727850,18.854352,92.000000
...,...,...,...,...,...,...,...,...
13653,2022-03-19,Fukushima Prefecture,1.747144e+07,5.076029e+02,37.470229,140.650578,34.527495,117.360489
13654,2022-03-19,Kagoshima Prefecture,2.956295e+05,5.022500e+02,31.833193,130.301922,20.687500,13.000000
13655,2022-03-19,Nara Prefecture,4.164816e+06,5.149348e+02,34.488300,136.165000,31.086957,535.000000
13656,2022-03-19,Shiga Prefecture,5.057522e+06,5.049167e+02,35.268280,136.249300,15.458333,100.000000


In [62]:
df = df.reset_index()

In [67]:
def plotly_plot(df):
    i = 0
    fig3 = go.Figure()
    country_list = list(df['Province'].unique())
    for country in country_list:
        fig3.add_trace(
            go.Scatter(
                x = df['Date'][df['Province']==country],
                y = df['Value'][df['Province']==country],
                name = country, visible = (i==0)
            )
        )

    fig3.update_layout(hovermode="x unified")
    fig3.update_layout(
        autosize=False,
        width=1000,
        height=800,)
    
    fig3.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)


    fig3.show()

In [68]:
plotly_plot(df)